## Setup

In [18]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## TidyCensus Example

This notebook grabs data from the US Census using the [tidycensus package in R](https://walker-data.com/tidycensus/index.html). Make sure you have installed the `tidyverse` and `tidycensus` packages.

In [20]:
%%R 

library(tidycensus)
library(tidyverse)

# store any shapefiles we grab from the census in ~/Library/Caches/tigris
# in case the government removes the shapefiles FTP server again 😭
options(tigris_use_cache = TRUE)

### Step 1 \| Load Census API key from `.env` file

The `.env` file should look like this:

```bash
CENSUS_API_KEY="your-api-key-here"
```

This is what is known as an environment variable. 

We load it below in Python

In [21]:
from dotenv import load_dotenv
load_dotenv() # <- searches for a file named .env and loads the environment variables in it

True

And in this cell we pass the environment variable to tidycensus in R

In [22]:
%%R 

census_api_key(Sys.getenv("CENSUS_API_KEY")) # <- Loads CENSUS_API_KEY from environment variable and sets it in R

To install your API key for use in future sessions, run this function with `install = TRUE`.


### Step 2 \| Figure out what tables you want data from and at what geographic granularity

Use <https://censusreporter.org/> to figure out which tables you want.

-   Scroll to the bottom of the page to see the tables.
-   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

Note, you probably always want population variable (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

- find some other variables that you're also interested in
- don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
).

👉 use the code from the example notebooks to get the variables you're interested in. feel free to use ChatGPT for help

In [ ]:
%%R 

library(sf)
library(tigris)

# TOPIC: Who are the super sloggers in NYC?
# Table B23020: Mean Usual Hours Worked for Workers 16-64 Years
#
# First, let's explore what variables are in this table

b23020_vars <- load_variables(2024, "acs5") %>%
  filter(str_detect(name, "B23020_"))

b23020_vars %>% 
  select(name, label) %>%
  print(n = 20)

In [ ]:
%%R 

# Fetch mean hours worked data at TRACT level
# B23020_001 = Mean hours for ALL workers
# B23020_002 = Mean hours for MALE workers  
# B23020_003 = Mean hours for FEMALE workers

nyc_hours <- get_acs(
  geography = "tract",
  state = "NY",
  county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
  variables = c(
    mean_hours_total = "B23020_001",
    mean_hours_male = "B23020_002",
    mean_hours_female = "B23020_003"
  ),
  year = 2024,
  survey = "acs5",
  geometry = TRUE,
  output = "wide"
)

# Quick look at the data
nyc_hours %>%
  st_drop_geometry() %>%
  select(NAME, mean_hours_totalE, mean_hours_maleE, mean_hours_femaleE) %>%
  head(10)

In [ ]:
%%R 

# Data is already wide format, just need to clean up and add borough

nyc_hours <- nyc_hours %>%
  mutate(
    borough = str_split(NAME, ",") %>% map_chr(2) %>% str_trim(),
    # Calculate gender gap in hours (male - female)
    hours_gap = mean_hours_maleE - mean_hours_femaleE,
    # Calculate CV for reliability check
    cv = ifelse(mean_hours_totalE > 0, 
                (mean_hours_totalM / mean_hours_totalE) * 100, NA)
  ) %>%
  filter(!is.na(mean_hours_totalE))

# Summary statistics
cat("MEAN HOURS WORKED - Summary:\n")
cat("============================\n\n")
summary(nyc_hours$mean_hours_totalE)

cat("\nBy gender:\n")
cat("Male mean:  ", round(mean(nyc_hours$mean_hours_maleE, na.rm=T), 1), "hours\n")
cat("Female mean:", round(mean(nyc_hours$mean_hours_femaleE, na.rm=T), 1), "hours\n")

cat("\nNumber of tracts:", nrow(nyc_hours))

In [ ]:
%%R -w 1000 -h 400

# Boxplot: Mean hours worked by borough
ggplot(nyc_hours) +
  aes(x = mean_hours_totalE, y = reorder(borough, mean_hours_totalE, FUN = median)) +
  geom_boxplot(fill = "steelblue", alpha = 0.7) +
  geom_vline(xintercept = 40, linetype = "dashed", color = "red") +
  annotate("text", x = 40.5, y = 0.7, label = "40hr standard", color = "red", hjust = 0) +
  labs(
    title = "Who Works the Longest Hours in NYC?",
    subtitle = "Mean usual hours worked per week by census tract",
    x = "Mean Hours Worked Per Week",
    y = ""
  ) +
  theme_minimal()

In [ ]:
%%R -w 1000 -h 400

# Who are the SUPER SLOGGERS? Find the top 10 tracts
top_sloggers <- nyc_hours %>%
  st_drop_geometry() %>%
  arrange(desc(mean_hours_totalE)) %>%
  select(NAME, borough, mean_hours_totalE, mean_hours_maleE, mean_hours_femaleE) %>%
  head(10)

cat("🏆 TOP 10 SUPER SLOGGER NEIGHBORHOODS:\n")
cat("======================================\n")
print(top_sloggers)

In [ ]:
%%R -w 1000 -h 1000

# Map: Where are the super sloggers?
nyc_hours_map <- nyc_hours %>% erase_water()

ggplot(nyc_hours_map) +
  geom_sf(aes(fill = mean_hours_totalE), color = NA) +
  scale_fill_viridis_c(
    option = "plasma",
    name = "Mean Hours\nWorked/Week",
    limits = c(30, 50)  # Cap the scale for better visibility
  ) +
  theme_void() +
  labs(
    title = "NYC's Super Sloggers: Where People Work the Longest Hours",
    subtitle = "Mean usual hours worked per week by census tract",
    caption = "Source: US Census Bureau, ACS 2020-2024 (Table B23020)"
  )

---

## Gender Gap in Hours Worked

Let's also look at the difference between male and female work hours. A positive gap means men work more hours; negative means women work more.

In [ ]:
%%R -w 1000 -h 400

# Gender gap by borough
ggplot(nyc_hours %>% filter(!is.na(hours_gap))) +
  aes(x = hours_gap, y = reorder(borough, hours_gap, FUN = median)) +
  geom_boxplot(fill = "purple", alpha = 0.7) +
  geom_vline(xintercept = 0, linetype = "dashed", color = "black") +
  labs(
    title = "Gender Gap in Work Hours by Borough",
    subtitle = "Positive = men work more hours; Negative = women work more",
    x = "Hours Gap (Male - Female)",
    y = ""
  ) +
  theme_minimal()

In [ ]:
%%R -w 1000 -h 1000

# Map: Gender gap in hours worked
ggplot(nyc_hours_map %>% filter(!is.na(hours_gap))) +
  geom_sf(aes(fill = hours_gap), color = NA) +
  scale_fill_gradient2(
    low = "purple",      # Women work more
    mid = "white",       # Equal
    high = "darkgreen",  # Men work more
    midpoint = 0,
    name = "Hours Gap\n(M - F)",
    limits = c(-10, 15)
  ) +
  theme_void() +
  labs(
    title = "Gender Gap in Work Hours Across NYC",
    subtitle = "Green = men work more hours; Purple = women work more hours",
    caption = "Source: US Census Bureau, ACS 2020-2024 (Table B23020)"
  )

---

## 📝 Key Findings

Run all the cells above to discover:

1. **Which neighborhoods have the longest work hours?** (Top 10 super sloggers)
2. **How do boroughs compare?** (Boxplot)
3. **Where are the super sloggers on the map?** (Choropleth)
4. **What's the gender gap in work hours?** (Male vs Female comparison)

### Data Quality Note

Unlike the birth data we analyzed earlier, **hours worked data is much more reliable** because:
- It's based on a larger population (all workers 16-64, not just women who gave birth)
- The estimates have lower margins of error
- Most tracts should have CV < 10%

---

## 📝 Key Takeaways: NYC's Super Sloggers

### What we found:

1. **Mean hours worked varies by neighborhood** - some census tracts average well above 40 hours/week
2. **Borough patterns** - Manhattan tends to have higher mean hours (finance, consulting, tech)
3. **Gender gap exists everywhere** - men work more hours on average in every borough
4. **Data quality is good** - unlike rare events (births), hours worked data is reliable at tract level

### Questions to explore further:

- Do high-hours neighborhoods correlate with high-income neighborhoods?
- Where is the gender gap largest? (Look for dark green on the gender gap map)
- Are there neighborhoods where women work MORE hours than men? (Purple areas)

---

*Table B23020: Mean Usual Hours Worked in the Past 12 Months for Workers 16 to 64 Years*